In [5]:
import numpy as np
import pandas as pd
import ast

In [243]:
movies_raw = pd.read_csv('dataset/movies.csv')
credits_raw = pd.read_csv('dataset/credits.csv')
ratings = pd.read_csv('dataset/ratings.csv')

In [244]:
movies = movies_raw.merge(credits_raw,on='title')
movies = movies[['movieId','genres','keywords','title','overview','cast','crew','tmdbId']]
movies.dropna(inplace=True)
movies.duplicated().sum()

0

In [245]:
'''
Content based generation
'''

'\nContent based generation\n'

In [246]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [247]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [248]:
def convertCast(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter < 3:
            L.append(i['name'])
            counter+=1
        else:
            break;
    return L

In [249]:
movies['cast'] = movies['cast'].apply(convertCast)

In [250]:
def fetchDirector(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break;
    return L

In [251]:
movies['crew'] = movies['crew'].apply(fetchDirector)

In [252]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [253]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [254]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [255]:
newDf = movies[['movieId','title','tags','tmdbId']]
newDf['tags'] = newDf['tags'].apply(lambda x: " ".join(x))
newDf['tags'] = newDf['tags'].apply(lambda x: x.lower())

C:\Users\otbho\AppData\Local\Temp/ipykernel_7944/1884961654.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newDf['tags'] = newDf['tags'].apply(lambda x: " ".join(x))
C:\Users\otbho\AppData\Local\Temp/ipykernel_7944/1884961654.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newDf['tags'] = newDf['tags'].apply(lambda x: x.lower())


In [256]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [257]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [258]:
newDf['tags'] = newDf['tags'].apply(stem)

C:\Users\otbho\AppData\Local\Temp/ipykernel_7944/877819958.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newDf['tags'] = newDf['tags'].apply(stem)


In [259]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vectors = cv.fit_transform(newDf['tags']).toarray()

In [260]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [261]:
def recommend(movieId):
    idx = newDf[newDf['movieId'] == movieId].index[0]
    distances = similarity[idx]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:11]
    L = []
    for i in movies_list:
        L.append(movies.iloc[i[0]].title)
    return L

In [262]:
recommend(21)

['The Lord of the Rings: The Two Towers',
 'The Hobbit: The Battle of the Five Armies',
 'The Lord of the Rings: The Fellowship of the Ring',
 'The Hobbit: An Unexpected Journey',
 'The Hobbit: The Desolation of Smaug',
 'Warcraft',
 'The Chronicles of Narnia: The Voyage of the Dawn Treader',
 'Brave',
 'Underworld: Rise of the Lycans',
 'The Scorpion King']

In [263]:
newDf

,movieId,title,tags,tmdbId
0,1,Inception,"cobb, a skill thief who commit corpor espionag...",27205
1,2,The Dark Knight,batman rais the stake in hi war on crime. with...,155
2,3,Avatar,"in the 22nd century, a parapleg marin is dispa...",19995
3,4,The Avengers,when an unexpect enemi emerg and threaten glob...,24428
4,5,Deadpool,deadpool tell the origin stori of former speci...,293660
...,...,...,...,...
1497,1496,xXx: State of the Union,"ice cube star as dariu stone, a thrill-seek tr...",11679
1498,1497,The Wolfman,"lawrenc talbot, an american man on a visit to ...",7978
1499,1498,Payback,"with friend like these, who need enemies? that...",2112
1500,1499,Child 44,"set in stalin-era soviet union, a disgrac mgb ...",181283


In [264]:
from joblib import dump
dump(similarity, 'model/content_based.joblib')
dump(newDf, 'model/movie_df.joblib')

['model/movie_df.joblib']

In [265]:
# ratings.to_csv('ratings.csv')

In [266]:
'''
Top Recommendation
'''

'\nTop Recommendation\n'

In [267]:

def weighted_rating(v,m,R,C):
    return ( (v / (v + m)) * R) + ( (m / (v + m)) * C )

In [268]:
# pre processing
vote_count = (
    ratings
    .groupby('movieId',as_index=False)
    .agg( {'userId':'count', 'rating':'mean'} )
    )
vote_count.columns = ['movieId', 'vote_count', 'avg_rating']

# calcuate input parameters
C = np.mean(vote_count['avg_rating'])
m = np.percentile(vote_count['vote_count'], 70)
vote_count = vote_count[vote_count['vote_count'] >= m]
R = vote_count['avg_rating']
v = vote_count['vote_count']
vote_count['weighted_rating'] = weighted_rating(v,m,R,C)

# post processing
vote_count = vote_count.merge(movies_raw, on = ['movieId'], how = 'left')
popular_items = vote_count.loc[:,['movieId','tmdbId','title','poster_path','backdrop_path','weighted_rating']]
popular_items = popular_items.sort_values('weighted_rating',ascending=False)
popular_items = popular_items.head(10)

In [269]:
from joblib import dump
dump(popular_items, 'model/top_rated.joblib')

['model/top_rated.joblib']

In [270]:
popular_items

,movieId,tmdbId,title,poster_path,backdrop_path,weighted_rating
1,2,155,The Dark Knight,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,/6fA9nie4ROlkyZAUlgKNjGNCbHG.jpg,4.686365
9,20,278,The Shawshank Redemption,/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg,/iNh3BivHyg5sQRPP1KOkzguEX0H.jpg,4.504546
10,21,122,The Lord of the Rings: The Return of the King,/rCzpDGLbOoPwLjy3OAm5NUPOTrC.jpg,/lXhgCODAbBXL5buk9yEmTpOoOgR.jpg,4.413637
7,16,120,The Lord of the Rings: The Fellowship of the Ring,/6oom5QYQ2yQTMJIbnvbkBL9cHo6.jpg,/vRQnzOn4HjIMX4LBq9nHhFXbsSu.jpg,4.379168
18,55,238,The Godfather,/3bhkrj58Vtu7enYsRolD1fZdja1.jpg,/AbgEQO2mneCSOc8CSnOMa8pBS8I.jpg,4.322728
5,13,624860,The Matrix,/8c4a8kE7PizaGQQnditMmI1xbRp.jpg,/eNI7PtK6DEYgZmHWP9gQNuff8pv.jpg,4.140910
4,6,157336,Interstellar,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,4.050001
0,1,27205,Inception,/edv5CZvWj09upOsy2Y6IwDhK8bt.jpg,/ztZ4vw151mw04Bg6rqJLQGBAmvn.jpg,3.962501
3,4,24428,The Avengers,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,/nNmJRkg8wWnRmzQDe2FwKbPIsJV.jpg,3.868183
2,3,19995,Avatar,/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg,/jlQJDD0L5ZojjlS0KYnApdO0n19.jpg,3.712501


In [271]:
'''
CF Recommendation
'''

'\nCF Recommendation\n'

In [272]:
genre = movies['genres']
# get all possible genre
all_genre = set()
for c in genre:
    for i in c:
        all_genre.update(c)
#     distinct_genre = genre[c].str.lower().str.strip().unique()
#     all_genre.update(distinct_genre)
# all_genre.remove(None)
all_genre = list(all_genre)

In [273]:
from joblib import dump
dump(all_genre, 'model/genres.joblib')

['model/genres.joblib']

In [274]:
# create item-genre matrix
item_genre_mat = movies[['movieId', 'genres']].copy()
item_genre_mat['genres'] = item_genre_mat['genres'].apply(lambda x: ''.join(x))

# OHE the genres column
for genre in all_genre:
    item_genre_mat[genre] = np.where(item_genre_mat['genres'].str.contains(genre), 1, 0)
item_genre_mat = item_genre_mat.drop(['genres'], axis=1)
item_genre_mat = item_genre_mat.set_index('movieId')

In [275]:
from joblib import dump
dump(item_genre_mat, 'model/item_genre_mat.joblib')

['model/item_genre_mat.joblib']

In [276]:
from scipy.sparse import csr_matrix
USER_COL = 'userId'
ITEM_COL = 'movieId'
RATING_COL = 'rating'
# preprocess data
row = ratings[USER_COL]
col = ratings[ITEM_COL]
data = ratings[RATING_COL]

# init user-item matrix
mat = csr_matrix((data, (row, col)))
mat.eliminate_zeros()

In [277]:

from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

NUM_USERS = 100
NUM_MOVIES = 1500


# preprocess data
row = ratings['userId']
col = ratings['movieId']
data = ratings['rating']

# init user-item matrix
mat = csr_matrix((data, (row, col)))
mat.eliminate_zeros()

# calculate sparsity
sparsity = float(len(mat.nonzero()[0]))
sparsity /= (mat.shape[0] * mat.shape[1])
sparsity *= 100
print(f'Sparsity: {sparsity:4.2f}%. This means that {sparsity:4.2f}% of the user-item ratings have a value.')

# compute similarity
item_corr_mat = cosine_similarity(mat.T)

Sparsity: 0.37%. This means that 0.37% of the user-item ratings have a value.


In [278]:
from joblib import dump
dump(item_corr_mat, 'model/cf_model.joblib')

['model/cf_model.joblib']

In [279]:
def top_k_items(item_id, top_k, corr_mat, map_name):
    # sort correlation value ascendingly and select top_k item_id
    top_items = corr_mat[item_id,:].argsort()[-top_k:][::-1] 
    top_items = [map_name[e] for e in top_items] 

    return top_items

In [280]:
# get top-k similar items
ind2name = {ind:name for ind,name in enumerate(item_genre_mat.index)}
name2ind = {v:k for k,v in ind2name.items()}

In [281]:
def getCfRecommendations(movieId):
#     if movie_cf_mat is None:      
#         r = open('model/cf_memory.pkl', 'rb')
#         movie_cf_mat = pickle.load(r)
#         print('loaded cf')
#         r.close()
    similar_items = top_k_items(name2ind[movieId],
                            top_k = 10,
                            corr_mat = item_corr_mat,
                            map_name = ind2name)
    if(movieId in similar_items):
        similar_items.remove(int(movieId))
    res = movies.loc[movies['movieId'].isin(similar_items)]
    return res

In [282]:
getCfRecommendations(16)

,movieId,genres,keywords,title,overview,cast,crew,tmdbId,tags
493,492,"[Drama, ScienceFiction]","[dystopia, blackandwhite, basedonyoungadultnovel]",The Giver,"[In, a, seemingly, perfect, community,, withou...","[JeffBridges, MerylStreep, BrentonThwaites]",[PhillipNoyce],227156,"[In, a, seemingly, perfect, community,, withou..."
494,493,"[Action, Adventure, Thriller]","[cia, armsdealer, sequel, rescuemission, hospi...",The Expendables 3,"[Barney,, Christmas, and, the, rest, of, the, ...","[SylvesterStallone, JasonStatham, HarrisonFord]",[PatrickHughes],138103,"[Barney,, Christmas, and, the, rest, of, the, ..."
495,494,"[ScienceFiction, Drama]","[moon, artificialintelligence, clone, isolatio...",Moon,"[With, only, three, weeks, left, in, his, thre...","[SamRockwell, KevinSpacey, DominiqueMcElligott]",[DuncanJones],580096,"[With, only, three, weeks, left, in, his, thre..."
496,495,"[History, Romance, War]","[nurse, patriotism, hawaii, worldwarii, pilot,...",Pearl Harbor,"[The, lifelong, friendship, between, Rafe, McC...","[BenAffleck, JoshHartnett, KateBeckinsale]",[MichaelBay],676,"[The, lifelong, friendship, between, Rafe, McC..."
497,496,[Drama],"[underdog, philadelphia, transporter, italo-am...",Rocky,"[When, world, heavyweight, boxing, champion,, ...","[SylvesterStallone, TaliaShire, BurtYoung]",[JohnG.Avildsen],1366,"[When, world, heavyweight, boxing, champion,, ..."
498,497,"[Adventure, Comedy, ScienceFiction]","[sandiego, area51, alienspacecraft, hitwithach...",Paul,"[For, the, past, 60, years,, a, space-travelin...","[SethRogen, SimonPegg, NickFrost]",[GregMottola],39513,"[For, the, past, 60, years,, a, space-travelin..."
499,498,"[Thriller, ScienceFiction, Action]","[darkness, dystopia, comet, alienlife-form, su...",Pitch Black,"[When, their, ship, crash-lands, on, a, remote...","[VinDiesel, RadhaMitchell, ColeHauser]",[DavidTwohy],2787,"[When, their, ship, crash-lands, on, a, remote..."
500,499,"[Action, Adventure, Crime]","[bomb, airport, bus, busride, highway, bombpla...",Speed,"[Los, Angeles, SWAT, cop, Jack, Traven, is, up...","[KeanuReeves, DennisHopper, SandraBullock]",[JandeBont],136797,"[Los, Angeles, SWAT, cop, Jack, Traven, is, up..."
502,501,"[Comedy, Action, ScienceFiction]","[endoftheworld, apocalypse, trilogy, homage]",The World's End,"[Five, friends, who, reunite, in, an, attempt,...","[SimonPegg, NickFrost, MartinFreeman]",[EdgarWright],107985,"[Five, friends, who, reunite, in, an, attempt,..."
1499,1498,"[Drama, Action, Thriller, Crime]","[newyork, heroin, money, criminal]",Payback,"[With, friends, like, these,, who, needs, enem...","[MelGibson, KrisKristofferson, GreggHenry]",[BrianHelgeland],2112,"[With, friends, like, these,, who, needs, enem..."


In [283]:
'''
Funk MF Model
'''

'\nFunk MF Model\n'

In [284]:
from surprise import SVD, accuracy
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate
from surprise.model_selection.split import train_test_split

# def pred2dict(predictions, top_k=None):
    
#     rec_dict = defaultdict(list)
#     for user_id, item_id, actual_rating, pred_rating, _ in predictions:
#         rec_dict[user_id].append((item_id, pred_rating))        
        
#     return rec_dict

# def get_top_k_recommendation(rec_dict, user_id, top_k, ind2name):
    
#     pred_ratings = rec_dict[user_id]
#     # sort descendingly by pred_rating
#     pred_ratings = sorted(pred_ratings, key=lambda x: x[1], reverse=True)
#     pred_ratings = pred_ratings[:top_k]
#     recs = [ind2name[e[0]] for e in pred_ratings]
    
#     return recs

# prepare train and test sets
reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(ratings, reader)
train, test = train_test_split(data, test_size=.2, random_state=42)

# init and fit the funk mf model
algo = SVD(random_state = 42)
algo.fit(train)
pred = algo.test(test);

# evaluation the test set
accuracy.rmse(pred)

# extract the item features from algo
movie_fmf_mat = cosine_similarity(algo.qi)

similar_items = top_k_items(name2ind[16],
                            top_k = 10,
                            corr_mat = movie_fmf_mat,
                            map_name = ind2name)

display(movies.loc[movies[ITEM_COL].isin(similar_items)])

# del movie_fmf_mat
# gc.collect();

RMSE: 1.0935


,movieId,genres,keywords,title,overview,cast,crew,tmdbId,tags
11,12,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...",The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],49026,"[Following, the, death, of, District, Attorney..."
15,16,"[Adventure, Fantasy, Action]","[elves, dwarves, orcs, middle-earth(tolkien), ...",The Lord of the Rings: The Fellowship of the Ring,"[Young, hobbit, Frodo, Baggins,, after, inheri...","[ElijahWood, IanMcKellen, CateBlanchett]",[PeterJackson],120,"[Young, hobbit, Frodo, Baggins,, after, inheri..."
57,58,"[Action, ScienceFiction]","[artificialintelligence, telepathy, intelligen...",Lucy,"[A, woman,, accidentally, caught, in, a, dark,...","[ScarlettJohansson, MorganFreeman, ChoiMin-sik]",[LucBesson],240832,"[A, woman,, accidentally, caught, in, a, dark,..."
99,100,"[Action, Thriller, ScienceFiction]","[suicide, assassin, drugaddiction, future, tim...",Looper,"[In, the, futuristic, action, thriller, Looper...","[JosephGordon-Levitt, BruceWillis, EmilyBlunt]",[RianJohnson],59967,"[In, the, futuristic, action, thriller, Looper..."
151,152,"[Adventure, Action, ScienceFiction]","[senate, investigation, army, deathstar, jedi,...",Star Wars: Episode II - Attack of the Clones,"[Ten, years, after, the, invasion, of, Naboo,,...","[EwanMcGregor, NataliePortman, HaydenChristensen]",[GeorgeLucas],1894,"[Ten, years, after, the, invasion, of, Naboo,,..."
221,222,"[Adventure, Action]","[savingtheworld, venice, holygrail, library, r...",Indiana Jones and the Last Crusade,"[When, Dr., Henry, Jones, Sr., suddenly, goes,...","[HarrisonFord, SeanConnery, DenholmElliott]",[StevenSpielberg],89,"[When, Dr., Henry, Jones, Sr., suddenly, goes,..."
246,247,"[Drama, Romance]","[individual, schizophrenia, massachusetts, lov...",A Beautiful Mind,"[At, Princeton, University,, John, Nash, strug...","[RussellCrowe, EdHarris, JenniferConnelly]",[RonHoward],453,"[At, Princeton, University,, John, Nash, strug..."
248,249,"[Crime, Drama, Thriller]","[basedonnovel, sniper, investigation, police, ...",Jack Reacher,"[In, an, innocent, heartland, city,, five, are...","[TomCruise, RosamundPike, RichardJenkins]",[ChristopherMcQuarrie],75780,"[In, an, innocent, heartland, city,, five, are..."
289,290,[Drama],"[individual, philosophy, poetry, shakespeare, ...",Dead Poets Society,"[At, an, elite,, old-fashioned, boarding, scho...","[RobinWilliams, EthanHawke, RobertSeanLeonard]",[PeterWeir],207,"[At, an, elite,, old-fashioned, boarding, scho..."
320,321,"[Adventure, Action]","[savingtheworld, riddle, whip, treasure, mexic...",Indiana Jones and the Kingdom of the Crystal S...,"[Set, during, the, Cold, War,, the, Soviets, –...","[HarrisonFord, CateBlanchett, ShiaLaBeouf]",[StevenSpielberg],217,"[Set, during, the, Cold, War,, the, Soviets, –..."


In [285]:
from joblib import dump
dump(movie_fmf_mat, 'model/funk_model.joblib')

['model/funk_model.joblib']

In [286]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import gc

# from collections import defaultdict

# # synthesize data
# NUM_USERS = 101
# NUM_ITEMS = 1501
# user_id = np.arange(start = 0, stop = NUM_USERS)
# item_id = np.arange(start = 0, stop = NUM_ITEMS)
# np.random.seed(42)

# user_item_dict = defaultdict(list)

# for id in user_id:
    
#     # random the number of item generation
#     # for each user, random 3 to 5 items to be rated.
#     num_rand_item = np.random.randint(low = 3, high = 8)

#     # random from the item_id
#     rand_items = np.random.choice(item_id, size = num_rand_item, replace = False)

#     # random rating for each itme_id
#     rand_rating = np.random.randint(low = 1, high = 5, size = num_rand_item)

#     # collect the user-item paris.
#     for uid, iid,rating in zip([id] * num_rand_item, rand_items, rand_rating):
#         user_item_dict['userid'].append(uid)
#         user_item_dict['movieId'].append(iid)
#         user_item_dict['rating'].append(rating)

# # prepare dataframe
# ratings = pd.DataFrame(user_item_dict)
# print("Rating Dataframe")
# ratings[['userid','movieId']] = ratings[['userid','movieId']].astype(str)
# display(ratings.head())

In [184]:
# ratings.to_csv('ratings_new.csv',index=False)

In [288]:
genre_dict = {}

In [322]:
for genre in all_genre:
    L = []
    count = 0
    for row in movies.itertuples():
        if count < 10 and genre in row[2]:
            L.append(row[8])
            count += 1
    genre_dict[genre] = L

In [323]:
 genre_dict

{'Fantasy': [19995, 70160, 120, 49051, 122, 121, 436969, 671, 209112, 22],
 'Western': [68718,
  281957,
  273248,
  57201,
  429,
  44896,
  44264,
  188161,
  5176,
  473831],
 'Family': [671, 14160, 150540, 20352, 10681, 177572, 12, 105, 585, 673],
 'Horror': [72190, 6479, 348, 694, 19908, 60304, 49849, 138843, 82654, 578],
 'TVMovie': [10947],
 'Thriller': [27205,
  155,
  76341,
  49026,
  135397,
  680,
  37724,
  597,
  16869,
  281957],
 'Crime': [155, 49026, 680, 278, 436969, 272, 106646, 238, 10528, 807],
 'Adventure': [27205,
  19995,
  24428,
  293660,
  157336,
  283995,
  70160,
  76341,
  68721,
  1726],
 'Romance': [13, 597, 82693, 102651, 152601, 64682, 222935, 38, 162, 8966],
 'ScienceFiction': [27205,
  19995,
  24428,
  157336,
  283995,
  70160,
  76341,
  624860,
  68721,
  1726],
 'Music': [114150,
  254470,
  3933,
  82695,
  224141,
  1584,
  277216,
  824,
  198277,
  10198],
 'Drama': [155, 157336, 68718, 550, 49026, 278, 13, 597, 272, 286217],
 'Comedy': [29

In [324]:
from joblib import dump
dump(genre_dict, 'model/genre_movies.joblib')

['model/genre_movies.joblib']

In [328]:
display(movies.loc[movies['tmdbId'].isin(genre_dict['Adventure'])])

,movieId,genres,keywords,title,overview,cast,crew,tmdbId,tags
0,1,"[Action, Thriller, ScienceFiction, Mystery, Ad...","[lossoflover, dream, kidnapping, sleep, subcon...",Inception,"[Cobb,, a, skilled, thief, who, commits, corpo...","[LeonardoDiCaprio, JosephGordon-Levitt, EllenP...",[ChristopherNolan],27205,"[Cobb,, a, skilled, thief, who, commits, corpo..."
2,3,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],19995,"[In, the, 22nd, century,, a, paraplegic, Marin..."
3,4,"[ScienceFiction, Action, Adventure]","[newyork, shield, marvelcomic, superhero, base...",The Avengers,"[When, an, unexpected, enemy, emerges, and, th...","[RobertDowneyJr., ChrisEvans, MarkRuffalo]",[JossWhedon],24428,"[When, an, unexpected, enemy, emerges, and, th..."
4,5,"[Action, Adventure, Comedy]","[antihero, mercenary, marvelcomic, superhero, ...",Deadpool,"[Deadpool, tells, the, origin, story, of, form...","[RyanReynolds, MorenaBaccarin, EdSkrein]",[TimMiller],293660,"[Deadpool, tells, the, origin, story, of, form..."
5,6,"[Adventure, Drama, ScienceFiction]","[savingtheworld, artificialintelligence, fathe...",Interstellar,"[Interstellar, chronicles, the, adventures, of...","[MatthewMcConaughey, JessicaChastain, AnneHath...",[ChristopherNolan],157336,"[Interstellar, chronicles, the, adventures, of..."
7,8,"[Action, ScienceFiction, Adventure]","[marvelcomic, spaceship, space, outerspace, or...",Guardians of the Galaxy,"[Light, years, from, Earth,, 26, years, after,...","[ChrisPratt, ZoeSaldana, DaveBautista]",[JamesGunn],283995,"[Light, years, from, Earth,, 26, years, after,..."
8,9,"[ScienceFiction, Adventure, Fantasy]","[hallucination, dystopia, femaleprotagonist, b...",The Hunger Games,"[Every, year, in, the, ruins, of, what, was, o...","[JenniferLawrence, JoshHutcherson, LiamHemsworth]",[GaryRoss],70160,"[Every, year, in, the, ruins, of, what, was, o..."
9,10,"[Action, Adventure, ScienceFiction, Thriller]","[future, chase, post-apocalyptic, dystopia, au...",Mad Max: Fury Road,"[An, apocalyptic, story, set, in, the, furthes...","[TomHardy, CharlizeTheron, NicholasHoult]",[GeorgeMiller],76341,"[An, apocalyptic, story, set, in, the, furthes..."
13,14,"[Action, Adventure, ScienceFiction]","[terrorist, waronterror, tennessee, malibu, ma...",Iron Man 3,"[When, Tony, Stark's, world, is, torn, apart, ...","[RobertDowneyJr., GwynethPaltrow, DonCheadle]",[ShaneBlack],68721,"[When, Tony, Stark's, world, is, torn, apart, ..."
14,15,"[Action, ScienceFiction, Adventure]","[middleeast, armsdealer, malibu, marvelcomic, ...",Iron Man,"[After, being, held, captive, in, an, Afghan, ...","[RobertDowneyJr., TerrenceHoward, JeffBridges]",[JonFavreau],1726,"[After, being, held, captive, in, an, Afghan, ..."
